In [3]:
import os
if os.path.basename(os.getcwd()) != 'biometria2':
    os.chdir(os.path.dirname(os.getcwd()))

In [30]:
from speech_embedder_net import SpeechEmbedder
from Speaker_Verification.utils import mfccs_and_spec
import torch
from hparam import hparam as hp

class SpekerVerifier:
    """Class for loading the model and computing speech embeddings"""
    def __init__(
            self,
            model_path: str = './speech_id_checkpoint/ckpt_epoch_360_batch_id_28.pth'
            ) -> None:
        self.model = SpeechEmbedder()
        self.model.load_state_dict(torch.load(model_path))
        self.model.eval()
        pass

    def compute_embedding(self, audio_input):
        _, mel_db, _ = mfccs_and_spec(audio_input, wav_process = True)
        mel_db = torch.Tensor(mel_db)
        enrollment_batch = torch.reshape(mel_db, (1, mel_db.size(0), mel_db.size(1)))
        embedding = self.model(torch.Tensor(enrollment_batch))
        return embedding

In [35]:
sample_file = 'split_data/train/id10001/00001.wav'
sample_file_same = 'split_data/train/id10001/00002.wav'
sample_file_diff = 'split_data/train/id10002/00002.wav'

model = SpekerVerifier()
embedding_org = model.compute_embedding(sample_file)
embedding_same = model.compute_embedding(sample_file_same)
embedding_diff = model.compute_embedding(sample_file_diff)

In [36]:
import torch.nn.functional as F

F.cosine_similarity(embedding_org, embedding_same)

tensor([0.9327], grad_fn=<SumBackward1>)

In [37]:
F.cosine_similarity(embedding_org, embedding_diff)

tensor([0.7410], grad_fn=<SumBackward1>)